In [1]:
from C3D_model import C3D

In [2]:
import torch
from torchvision import models
import torch.nn as nn
from torchvision import transforms, datasets
import os
import torch.nn.functional as F
import torch.optim as optim
import time

In [3]:
import numpy as np
import cv2

import time
from __future__ import print_function

import torch
from torch.autograd import Variable

from os.path import join
from glob import glob

import skimage.io as io
from skimage.transform import resize

from C3D_model import C3D

In [4]:
from scipy.io import loadmat

In [5]:
model = C3D()
model.load_state_dict(torch.load('c3d.pickle'))

In [6]:
feature_extractor = torch.nn.Sequential(*(list(model.children())[:-5]))

In [7]:
list(model.children())[:-6]

[Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False),
 Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False),
 Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False),
 Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False),
 Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),
 Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1)),


In [8]:
list(model.children())[13]

Linear(in_features=8192, out_features=4096, bias=True)

In [9]:
feature_extractor = torch.nn.Sequential(*(list(model.children())[:-6]))
feature_extractor_head = torch.nn.Sequential((list(model.children())[13]))

In [10]:
def conv_dict(dict2):
    i = 0
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict

In [11]:
def get_weight(weight_path, layer):
    # To load the weights from file
    dict2 = loadmat(weight_path)
    weights = conv_dict(dict2)
    # TO get the required weight
    weight = np.array(weights[layer])
    shape = weight[0].shape
    weight = np.reshape(weight[0], (shape[1], shape[0]))
    weight = torch.tensor(weight)
    return weight

In [12]:
class anomaly_ann(nn.Module):
    def __init__(self):
        super(anomaly_ann, self).__init__()
        weights = './weights_L1L2.mat'
        self.layer1 = nn.Linear(4096, 512)
        self.layer1.weight.data = get_weight(weights, '0')

        self.layer2 = nn.Linear(512, 32)
        self.layer2.weight.data = get_weight(weights, '2')

        self.layer3 = nn.Linear(32, 1)
        self.layer3.weight.data = get_weight(weights, '4')
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        #print("x", x.shape)
        out = self.layer1(x)
        out = self.relu(out)
        
        #out = (out-out.mean())/(out.max()-out.mean())
        #print("after vgg", out.shape)
        out = self.layer2(out)
        #out = (out-out.mean())/(out.max()-out.mean())
        #print("after transpose", out.shape)
        out = self.layer3(out)
        #print("after output layer", out.shape)
        print(out)
        return self.sigmoid(out)

In [13]:
class anomaly_detector(nn.Module):
    def __init__(self):
        super(anomaly_detector, self).__init__()
        self.feature_extractor = feature_extractor
        self.feature_extractor_head = feature_extractor_head
        self.ann = anomaly_ann()
        
    def forward(self, x):
        #print("x", x.shape)
        out = self.feature_extractor(x)
        
        out = out.view(-1, 8192)
        out = self.feature_extractor_head(out)
        out = self.ann(out)
        return out

In [14]:
net = anomaly_detector()

In [15]:
net

anomaly_detector(
  (feature_extractor): Sequential(
    (0): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (6): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (8): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (9): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (10): Conv3d(512, 512, kernel_s

In [16]:
def get_blocc(vid):
    arr = []
    for i in range(16):
        arr.append(vid.read()[1])

    blocc = np.array([resize(frame, output_shape=(112, 200), preserve_range=True) for frame in arr])

    blocc = blocc[:, :, 44:44+112, :]
    blocc = blocc.transpose(3, 0, 1, 2)  # ch, fr, h, w
    blocc = np.expand_dims(blocc, axis=0)  # batch axis
    blocc = np.float32(blocc)
    blocc = torch.from_numpy(blocc)
    
    return blocc

In [17]:
vid_file = '/home/nevin/RoadAccidents010_x264.mp4'

In [18]:
vid = cv2.VideoCapture(vid_file)

In [19]:
while(vid):
    X = get_blocc(vid)
    X = Variable(X)

    start_time = time.time()

    prediction = net(X)
    prediction = prediction.data.cpu().numpy()

    print(prediction)

/home/nevin/anaconda3/envs/pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/nevin/anaconda3/envs/pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackward>)
[[0.5502221]]
tensor([[0.2016]], grad_fn=<ThAddmmBackw

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# With Learnable Parameters
m = nn.BatchNorm1d(100)
# Without Learnable Parameters
m = nn.BatchNorm1d(100, affine=False)
input = torch.randn(20, 100)
output = m(input)


In [ ]:
X

In [ ]:
get_blocc(vid)

In [ ]:
s.mean()

In [ ]:
s/s.mean()